In [2]:
# Step 1: Import Libraries

import os
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
from scipy.stats import mode

# Set random seed for reproducibility
np.random.seed(42)


In [3]:
# Get the directory of the current script
SCRIPT_DIR = os.getcwd()

# Define data directories
DATA_DIR = os.path.join(SCRIPT_DIR, "..", "data", "processed")
MODEL_DIR = os.path.join(SCRIPT_DIR, "..", "models")

# Load datasets with optimized settings
def load_data(filename):
    filepath = os.path.join(DATA_DIR, filename)
    return pd.read_csv(filepath, parse_dates=["Start date"], index_col="Start date", low_memory=False)

df_hourly = load_data("processed_hourly_data.csv")
df_daily = load_data("processed_daily_data.csv")
df_weekly = load_data("processed_weekly_data.csv")

# ✅ Ensure data is sorted for time series analysis
df_hourly.sort_index(inplace=True)
df_daily.sort_index(inplace=True)
df_weekly.sort_index(inplace=True)

print("✅ Processed data loaded and sorted successfully!")


✅ Processed data loaded and sorted successfully!


In [4]:
# Define price columns
PRICE_COLUMNS = [
    "Germany/Luxembourg [/MWh] Original resolutions",
    "Belgium [/MWh] Original resolutions",
    "France [/MWh] Original resolutions"
]

# Compute the average price across regions
for df in [df_hourly, df_daily, df_weekly]:
    df["Average_Price_€/MWh"] = df[PRICE_COLUMNS].mean(axis=1)

# 🔄 Price Direction Labels (+1 = Up, -1 = Down, 0 = Stable)
def compute_direction(price_series, threshold=0.05):
    """Compute price movement direction based on percentage change."""
    price_change = price_series.pct_change().fillna(0)
    return np.select(
        [price_change > threshold, price_change < -threshold],
        [1, -1], 
        default=0
    )

# Apply function to all datasets
for df in [df_hourly, df_daily, df_weekly]:
    df["Price_Direction"] = compute_direction(df["Average_Price_€/MWh"]).astype(int)

print("✅ Feature Engineering Completed!")


✅ Feature Engineering Completed!


C:\Users\Elite\AppData\Local\Temp\ipykernel_12760\1313747541.py:15: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  price_change = price_series.pct_change().fillna(0)


In [5]:
# 🎯 Define Target & Labels
TARGET = "Average_Price_€/MWh"
LABEL = "Price_Direction"

# 🔄 Ensure datasets are sorted by date
for df in [df_hourly, df_daily, df_weekly]:
    df.sort_index(inplace=True)

# 📊 Rolling Mean Features (Smoother Trends)
df_hourly["Rolling_Mean_24h"] = df_hourly[TARGET].rolling(window=24, min_periods=1).mean()
df_daily["Rolling_Mean_7d"] = df_daily[TARGET].rolling(window=7, min_periods=1).mean()
df_weekly["Rolling_Mean_4w"] = df_weekly[TARGET].rolling(window=4, min_periods=1).mean()

# ⚡️ Lag Features
df_hourly["Lag_1h"] = df_hourly[TARGET].shift(1)
df_hourly["Lag_24h"] = df_hourly[TARGET].shift(24)

# 📈 Volatility (Rolling Std Dev)
df_hourly["Volatility_24h"] = df_hourly[TARGET].rolling(window=24, min_periods=1).std()

# 🔄 Percentage Change
df_hourly["Price_Change_1h"] = df_hourly[TARGET].pct_change(1) * 100
df_hourly["Price_Change_24h"] = df_hourly[TARGET].pct_change(24) * 100

# 🚀 Fill NaNs After Rolling/Lag Operations
for df in [df_hourly, df_daily, df_weekly]:
    df.fillna(method="bfill", inplace=True)  # Use backward fill to retain trends

# ✅ Expected Feature Sets
FEATURES_HOURLY = ["Rolling_Mean_24h", "Lag_1h", "Lag_24h", "Volatility_24h", "Price_Change_1h", "Price_Change_24h"]
FEATURES_DAILY = ["Rolling_Mean_7d"]
FEATURES_WEEKLY = ["Rolling_Mean_4w"]

# 📊 Feature Selection
X_hourly, y_hourly = df_hourly[FEATURES_HOURLY], df_hourly[LABEL]
X_daily, y_daily = df_daily[FEATURES_DAILY], df_daily[LABEL]
X_weekly, y_weekly = df_weekly[FEATURES_WEEKLY], df_weekly[LABEL]

# 🔍 Debugging: Check for Issues in Feature Matrices
for name, X in [("Hourly", X_hourly), ("Daily", X_daily), ("Weekly", X_weekly)]:
    if X.isna().sum().sum() > 0:
        print(f"⚠ NaN values found in {name} dataset")
    if np.isinf(X).sum().sum() > 0:
        print(f"⚠ Infinite values found in {name} dataset")

# 🔧 Normalize Features for LSTM
scaler = MinMaxScaler()
X_hourly_scaled = scaler.fit_transform(X_hourly)
X_daily_scaled = scaler.fit_transform(X_daily)
X_weekly_scaled = scaler.fit_transform(X_weekly)

# 📚 Train-Test Split
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X_hourly_scaled, y_hourly, test_size=0.2, random_state=42)
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_daily_scaled, y_daily, test_size=0.2, random_state=42)
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_weekly_scaled, y_weekly, test_size=0.2, random_state=42)

# 🔄 Reshape for LSTM (samples, time steps, features)
X_train_h_lstm = X_train_h.reshape((X_train_h.shape[0], 1, X_train_h.shape[1]))
X_test_h_lstm = X_test_h.reshape((X_test_h.shape[0], 1, X_test_h.shape[1]))

X_train_d_lstm = X_train_d.reshape((X_train_d.shape[0], 1, X_train_d.shape[1]))
X_test_d_lstm = X_test_d.reshape((X_test_d.shape[0], 1, X_test_d.shape[1]))

X_train_w_lstm = X_train_w.reshape((X_train_w.shape[0], 1, X_train_w.shape[1]))
X_test_w_lstm = X_test_w.reshape((X_test_w.shape[0], 1, X_test_w.shape[1]))

print("✅ Data Preparation Completed!")


✅ Data Preparation Completed!


C:\Users\Elite\AppData\Local\Temp\ipykernel_12760\2448073711.py:22: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_hourly["Price_Change_1h"] = df_hourly[TARGET].pct_change(1) * 100
C:\Users\Elite\AppData\Local\Temp\ipykernel_12760\2448073711.py:23: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_hourly["Price_Change_24h"] = df_hourly[TARGET].pct_change(24) * 100
C:\Users\Elite\AppData\Local\Temp\ipykernel_12760\2448073711.py:27: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="bfill", in

In [6]:
# 🛠 Step 1: Fix `Price_Direction` Computation
def compute_direction(prices, threshold=0.05):
    """
    Computes price direction based on percentage change.
    Returns: 1 (rise), -1 (fall), 0 (stable)
    """
    price_change = prices.pct_change().fillna(0)
    return np.where(price_change > threshold, 1, np.where(price_change < -threshold, -1, 0))

df_hourly["Price_Direction"] = compute_direction(df_hourly["Average_Price_€/MWh"], threshold=0.05)
df_hourly["Price_Direction"] = df_hourly["Price_Direction"].astype(int)

# 🎯 Target variable
y = df_hourly["Price_Direction"]

# 🏗 Step 2: Define Features (Drop unnecessary columns)
X = df_hourly.drop(columns=["Timestamp", "Price_Direction"], errors="ignore")

# 🛠 Step 3: Handle NaN & Infinite Values
X.replace([np.inf, -np.inf], np.nan, inplace=True)  # Convert infinities to NaN
X.fillna(method="bfill", inplace=True)  # Backfill NaNs

# 🏗 Step 4: Split Data
X_train_h, X_test_h, y_train_h, y_test_h = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 🔄 Adjust Labels to Start from 0
label_map = {-1: 0, 0: 1, 1: 2}
y_train_h = y_train_h.replace(label_map)
y_test_h = y_test_h.replace(label_map)

# 🛠 Step 5: Handle Class Imbalance (SMOTE)
if len(np.unique(y_train_h)) == 1:
    print("⚠ Warning: Only one class detected! Applying SMOTE to balance classes.")
    smote = SMOTE(random_state=42)
    X_train_h, y_train_h = smote.fit_resample(X_train_h, y_train_h)

# 🔎 Step 6: Ensure Consistency Between Train & Test Features
X_test_h = X_test_h[X_train_h.columns]  # Align test features with train set

# ✅ Debugging: Check Class Distribution
print("📊 Class Distribution After SMOTE:")
print(pd.Series(y_train_h).value_counts())


📊 Class Distribution After SMOTE:
Price_Direction
1    11807
2     1983
0     1454
Name: count, dtype: int64


C:\Users\Elite\AppData\Local\Temp\ipykernel_12760\2547866890.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X.fillna(method="bfill", inplace=True)  # Backfill NaNs


In [9]:
# 🏆 Step 1: Initialize Models
lgb_model = lgb.LGBMClassifier(random_state=42, n_estimators=500, learning_rate=0.05, force_col_wise=True)
xgb_model = xgb.XGBClassifier(random_state=42, n_estimators=500, learning_rate=0.05, use_label_encoder=False)

# 🚀 Step 2: Fix Feature Names
X_train_h.columns = [col.replace(" ", "_").replace("{", "").replace("}", "").replace("[", "")
                     .replace("]", "").replace(":", "").replace("\\", "").replace("\"", "")
                     for col in X_train_h.columns]
X_test_h.columns = X_train_h.columns  # Ensure test set has the same feature names

# ✅ Step 3: Train Models
lgb_model.fit(X_train_h, y_train_h)
xgb_model.fit(X_train_h, y_train_h)

# 🔮 Step 4: Predictions
y_pred_lgb = lgb_model.predict(X_test_h)
y_pred_xgb = xgb_model.predict(X_test_h)

# 📊 Step 5: Evaluation
print(f"✅ LightGBM Accuracy: {accuracy_score(y_test_h, y_pred_lgb):.4f}")
print(f"✅ XGBoost Accuracy: {accuracy_score(y_test_h, y_pred_xgb):.4f}")


[LightGBM] [Info] Total Bins 21913
[LightGBM] [Info] Number of data points in the train set: 15244, number of used features: 90
[LightGBM] [Info] Start training from score -2.349868
[LightGBM] [Info] Start training from score -0.255493
[LightGBM] [Info] Start training from score -2.039575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

c:\Users\Elite\Documents\ocean\PowerCast The Electricity Price Forecasting Challenge\PowerCast_v2\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [13:49:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ LightGBM Accuracy: 0.9976
✅ XGBoost Accuracy: 0.9955


In [10]:
# 🏆 Step 1: Majority Voting for Ensemble Prediction
ensemble_preds = np.round((y_pred_lgb + y_pred_xgb) / 2).astype(int)  # Adjusted for binary/multi-class

# ✅ Step 2: Evaluate Ensemble Accuracy
ensemble_accuracy = accuracy_score(y_test_h, ensemble_preds)
print(f"📊 Ensemble Model Accuracy: {ensemble_accuracy:.4f}")

# 🔍 Step 3: Volatility Capture (Standard Deviation Comparison)
actual_volatility = y_test_h.diff().std()
predicted_volatility = pd.Series(ensemble_preds).diff().std()
volatility_score = 1 - abs(actual_volatility - predicted_volatility) / actual_volatility if actual_volatility != 0 else 0

print(f"📈 Actual Volatility: {actual_volatility:.4f}")
print(f"🔮 Predicted Volatility: {predicted_volatility:.4f}")
print(f"📊 Volatility Capture Score: {volatility_score:.4f}")

# ⚠️ Step 4: Extreme Price Movement Detection (Changes > 15%)
y_test_h = pd.Series(y_test_h)
extreme_moves = (y_test_h.abs() > 0.15).sum()
extreme_correct = ((y_test_h.abs() > 0.15) & (np.abs(ensemble_preds) > 0.15)).sum()
extreme_accuracy = extreme_correct / extreme_moves if extreme_moves > 0 else 0

print(f"🚀 Extreme Price Movement Accuracy: {extreme_accuracy:.4f}")


📊 Ensemble Model Accuracy: 0.9974
📈 Actual Volatility: 0.6690
🔮 Predicted Volatility: 0.6701
📊 Volatility Capture Score: 0.9982
🚀 Extreme Price Movement Accuracy: 0.9988


In [ ]:
def evaluate_model(y_true, y_pred, price_series, test_indices):
    """
    Evaluates the model using multiple performance metrics.

    Args:
        y_true (array-like): True class labels.
        y_pred (array-like): Predicted class labels.
        price_series (pd.Series): Full price series to compute extreme movements.
        test_indices (array-like): Indices corresponding to y_test_h.
    """

    # Convert to NumPy arrays
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()

    # ✅ 1. Directional Accuracy
    directional_acc = accuracy_score(y_true, y_pred)

    # 📉 2. Mean Absolute Error (MAE) & Root Mean Squared Error (RMSE)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # 📊 3. Volatility Capture (Compare Standard Deviations)
    true_volatility = np.std(y_true)
    pred_volatility = np.std(y_pred)
    volatility_capture = 1 - abs(true_volatility - pred_volatility) / true_volatility if true_volatility != 0 else 0

    # ⚠️ 4. Extreme Price Movement Accuracy (> 15% actual price change)
    price_pct_change = price_series.pct_change().fillna(0)  # Compute full dataset % change

    # 🔍 Subset price percentage change for test set using `.loc[]`
    test_price_pct_change = price_pct_change.loc[test_indices].reset_index(drop=True)

    # Ensure matching lengths
    if len(test_price_pct_change) != len(y_pred):
        raise ValueError(f"Mismatch in lengths: price_pct_change ({len(test_price_pct_change)}) vs y_pred ({len(y_pred)})")

    # Compute extreme movement accuracy
    extreme_moves = (test_price_pct_change.abs() > 0.15).sum()
    extreme_correct = ((test_price_pct_change.abs() > 0.15) & (y_pred != 1)).sum()
    extreme_accuracy = extreme_correct / extreme_moves if extreme_moves > 0 else 0

    # 🔥 5. Print Evaluation Metrics
    print(f"✅ Directional Accuracy: {directional_acc:.4f}")
    print(f"📉 Mean Absolute Error (MAE): {mae:.4f}")
    print(f"📊 Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"📈 Volatility Capture Score: {volatility_capture:.4f}")
    print(f"⚠️ Extreme Price Movement Accuracy: {extreme_accuracy:.4f}")

# 🔎 Run Evaluation
evaluate_model(y_test_h, ensemble_preds, df_hourly["Average_Price_€/MWh"], test_indices=y_test_h.index)

✅ Directional Accuracy: 0.9974
📉 Mean Absolute Error (MAE): 0.0026
📊 Root Mean Squared Error (RMSE): 0.0512
📈 Volatility Capture Score: 0.9975
⚠️ Extreme Price Movement Accuracy: 1.0000


In [13]:
# Define model save directory
MODEL_DIR = "../models"
os.makedirs(MODEL_DIR, exist_ok=True)

# Save trained models
joblib.dump(lgb_model, os.path.join(MODEL_DIR, "lightgbm_model.pkl"))
joblib.dump(xgb_model, os.path.join(MODEL_DIR, "xgboost_model.pkl"))

print("✅ Models saved successfully in 'models/' directory!")


✅ Models saved successfully in 'models/' directory!
